# ***Question 2 - Biggest strides in decreasing CO<sub>2</sub>***

At first I'm importing the required packages so I can write my code clearly.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb as ddb

Here I'm getting the needed data from a raw data source, with checking if the data loads like it should. 

By using the raw data source it is readable not only by me and also when the data updates, I can do the same analysis with the updated information without the need to download new files, this improves my efficiency in workflow.

I've chosen this data source because these are relations that are commonly announced in media and other information sources.

In [ ]:
# import data
df_co2 = pd.read_csv("https://ourworldindata.org/grapher/annual-co2-emissions-per-country.csv?v=1&csvType=full&useColumnShortNames=false")
df_pop = pd.read_csv("https://ourworldindata.org/grapher/population.csv?v=1&csvType=full&useColumnShortNames=true")
df_co2_pc = pd.read_csv("https://ourworldindata.org/grapher/co2-emissions-per-capita.csv?v=1&csvType=full&useColumnShortNames=false")
# df_co2.head()
# df_pop.head()
# df_co2_pc.head()

I'm selecting 2 years 10 years apart for this analysis to rule out coincidences. A longer term will probably yield more robust conclusions.

I'm using duckdb (ddb) because it makes selecting the required years easy with a syntax I'm quite familiar with.

In [ ]:
df_co2 = ddb.query("""
    SELECT *
    FROM df_co2
    where year in (2013, 2023)
""").to_df()

In this tab I will change the names of the columns, to improve the workflow.

I'm also merging the different dataframes on 'code' and 'year', I do this to work with on a single dataframe.


In [ ]:
# Rename columns
df_co2.rename(columns={'Annual CO₂ emissions': "CO2"}, inplace=True)
df_pop.rename(columns={'population_historical': "Pop"}, inplace=True)
df_co2_pc.rename(columns={'Annual CO₂ emissions (per capita)': "CO2_pc"}, inplace=True)

# Merge dataframes
df_merged = pd.merge(df_co2, df_pop.drop(columns = ['Entity']) ,how='left', on=['Code', 'Year'])
df_merged = pd.merge(df_merged, df_co2_pc.drop(columns = ['Entity']), how='left', on=['Code', 'Year'])
df_final = df_merged.copy()
df_final.head()

Per cap means 'CO<sub>2</sub>' / 'pop'

In [ ]:
df_final['change_co2_per_cap'] = df_final.groupby('Entity')['CO2_pc'].diff()

df_final.head(200)


I'm ruling out the rows with missing values so I can work wit a clean dataset, missing values could affect my conclusion because of missing datapoints.

In [ ]:
df_final = df_final[df_final['Code'].notna()]

I've added a percentage change column to see the relative change in emission.

In [ ]:
#Calculate percentage change in CO2 emission and population
# Use 2013 values (first value in each group) as the baseline
df_final['CO2_pc_2013'] = df_final.groupby('Entity')['CO2_pc'].transform('first')
df_final['% Change Emission'] = round(((df_final['change_co2_per_cap'] / df_final['CO2_pc_2013'])*100),2)

df_final.head(2000)


Now i have cleaned my dataset from missing values by selecting only 2023.

In [ ]:
df_final = df_final[df_final['Year'].eq(2023)]

For the final analysis I'm creating a graph to see which country has the highest decrease in CO<sub>2</sub> output per capita to eliminate population size effects.

In [ ]:
# df = sns.load_dataset('df_final22')
top10 = df_final.sort_values(by='% Change Emission', ascending=True).head(10)
ax = sns.barplot(x = '% Change Emission',y = 'Entity',data = top10)
plt.xticks(rotation=60)
# ax.bar_label(ax.containers[0])
plt.title('Top 10 countries with the biggest strides in decreasing CO2 per capita')
plt.xlabel('% CO2 emission change in last decade (2013-2023)')
plt.ylabel('Country')
plt.show()

As you can see above, Yemen has the largest decrease in capita in the last decade.

# ***conclusion***

Yemen makes the biggest strides in decreasing CO<sub>2</sub> when I look to the last decade.